In [1]:
# Import Libraries
import requests
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
# The List of cryptocurrencies
coin_list = ['BTC']

In [3]:
#defining the dataframe
main_df = pd.DataFrame()

In [4]:
# Reset datetime_end for each coin to ensure the loop starts with the current date and time
datetime_end = datetime.now()
for coin in coin_list:
    coin_df = pd.DataFrame()  # Initialize an empty DataFrame for each coin
    
    while True:
        # Define the time range (12-hour intervals)
        datetime_start = datetime_end - relativedelta(hours=12)
        
        # API for scraping
        url = f'https://production.api.coindesk.com/v2/price/values/{coin}?start_date={datetime_start.strftime("%Y-%m-%dT%H:%M")}&end_date={datetime_end.strftime("%Y-%m-%dT%H:%M")}&ohlc=true'
        
        # Fetch data from the API
        temp_data = requests.get(url).json()
        
        # Convert the fetched data to a DataFrame
        df = pd.DataFrame(temp_data['data']['entries'])
        
        # Check if the DataFrame is empty before proceeding
        if df.empty:
            print(f"No more data for {coin} in this time range.")
            break
        
        # Assign column names
        df.columns = ['Timestamp', 'Open', 'High', 'Low', 'Close']
        
        # Handle missing data logic as needed (not provided)
        
        # Process timestamps and append to the coin_df
        df = df.drop(['Timestamp'], axis=1)  # Drop the raw timestamp column
        df['Datetime'] = [datetime_end - relativedelta(minutes=len(df)-i) for i in range(0, len(df))]  # Add a 'Datetime' column
        coin_df = pd.concat([coin_df, df], ignore_index=True)  # Concatenate df to coin_df

        # Move the datetime_end backwards by 12 hours
        datetime_end = datetime_start
    
    # Add the cryptocurrency symbol to the DataFrame
    if not coin_df.empty:
        coin_df['Symbol'] = coin
    
    # Concatenate the coin_df to the main_df
    main_df = pd.concat([main_df, coin_df], ignore_index=True)

# Export the final dataframe to a CSV file
main_df.to_csv('BTC_df.csv', index=False)

No more data for BTC in this time range.


In [5]:
main_df = main_df[['Datetime', 'Symbol', 'Open', 'High', 'Low', 'Close']].reset_index(drop=True)
main_df

,Datetime,Symbol,Open,High,Low,Close
0,2024-10-21 12:50:24.917752,BTC,68672.876640,68673.430524,68658.638010,68663.384456
1,2024-10-21 12:51:24.917752,BTC,68664.134150,68664.134150,68645.616929,68650.190862
2,2024-10-21 12:52:24.917752,BTC,68647.202306,68655.503119,68624.012529,68625.220021
3,2024-10-21 12:53:24.917752,BTC,68624.876797,68644.715017,68620.710033,68639.687558
4,2024-10-21 12:54:24.917752,BTC,68632.261842,68632.261842,68613.203077,68616.455362
...,...,...,...,...,...,...
3099,2024-08-22 07:17:24.917752,BTC,60846.819307,60849.493597,60662.457478,60686.464231
3100,2024-08-22 07:18:24.917752,BTC,60744.390989,60966.230079,60711.616098,60735.623447
3101,2024-08-22 07:19:24.917752,BTC,60778.311319,60881.715402,60709.936587,60877.832835
3102,2024-08-21 19:18:24.917752,BTC,59989.928759,60689.146572,59833.749692,60377.391520


In [6]:
main_df.to_csv('EOS_df.csv', index=False)